In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
import openai
openai.api_key = "sk-xxx"

In [ ]:
from pathlib import Path
from llama_index import download_loader, VectorStoreIndex, load_index_from_storage
from llama_index.storage.storage_context import StorageContext

PDFReader = download_loader("PDFReader")

loader = PDFReader()

class PodcastTitle:
    def __init__(self, name, about, file, key):
        self.name = name
        self.about = about   
        self.file = file
        self.key = key

podcast_titles = [
    PodcastTitle("10 Tools for Managing Stress and Anxiety with Huberman", "manage stress and anxiety", "behaviour/10_Tools_for_Managing_Stress_&_Anxiety_Huberman_Lab_Podcast_10.pdf", "tools_for_stress"),
    PodcastTitle("The Science of Setting and Achieving Goals with Huberman", "set and achieve goals", "behaviour/55_The_Science_of_Setting_&_Achieving_Goals_Huberman_Lab_Podcast_55.pdf", "setting_goals"),
    PodcastTitle("Dr Chris Palmer Diet and Nutrition for Mental Health with Huberman", "have healthy diet for mental health", "food/99_Dr_Chris_Palmer_Diet_&_Nutrition_for_Mental_Health_Huberman_Lab_Podcast_99.pdf", "diet_nutrition"),
]

podcast_vector_index = {}
for podcast in podcast_titles:
    try:
        storage_context = StorageContext.from_defaults(persist_dir=f"./storage/cache/{podcast.key}_vector")
        podcast_vector_index[podcast.key] = load_index_from_storage(storage_context)
    except:
        documents = loader.load_data(file=Path(f"./assets/AndrewHuberman/{podcast.file}"))
        vector_index = VectorStoreIndex.from_documents(documents)
        podcast_vector_index[podcast.key] = vector_index
        vector_index.storage_context.persist(persist_dir=f"./storage/cache/{podcast.key}_vector")




In [4]:
podcast_vector_index

{'tools_for_stress': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x1453c2070>,
 'setting_goals': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x12b0c2340>,
 'diet_nutrition': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x14561e6d0>}

In [7]:
from llama_index.indices.postprocessor import KeywordNodePostprocessor
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata


node_processor = KeywordNodePostprocessor(
    exclude_keywords=["supplements", "LMNT", "InsideTracker", "Helix", "ROKA", "Athletic Greens", "Thesis", "Eight Sleep"]
)

query_engine_tools = []
podcats_vector_engines = {}

for podcast in podcast_titles:
    retriever = VectorIndexRetriever(
        index = podcast_vector_index[podcast.key],
        similarity_top_k=3,
    )

    podcats_vector_engines[podcast.key] = RetrieverQueryEngine(
        retriever = retriever,
        node_postprocessors=[node_processor]
    )

    new_tool = QueryEngineTool(
        query_engine=podcats_vector_engines[podcast.key],
        metadata = ToolMetadata(
            name=f"{podcast.key}_vector_tool",
            description=f"Useful for retrieving specific context from a podcast {podcast.name}. "
            f"Use when you need information related to {podcast.about}.",
        )

    )
    query_engine_tools.append(new_tool)

In [9]:
query_engine_tools

In [10]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

agent = OpenAIAgent.from_tools(query_engine_tools, llm=OpenAI(temperature=0, model="gpt-3.5-turbo-0613"), verbose=True)

In [11]:
openai.log = "debug"

In [15]:
response = agent.chat("Tell me about Ketogenic diet")

message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "user", "content": "Tell me about setting and achieving goals"}, {"role": "assistant", "content": null, "function_call": {"name": "setting_goals_vector_tool", "arguments": "{\\n  \\"input\\": \\"setting and achieving goals\\"\\n}"}}, {"role": "function", "content": "Setting and achieving goals involves several key factors. First, it is important to set moderate goals that are challenging but possible to achieve. It is also important to limit the number of goals pursued at once, as pursuing too many goals can be counterproductive. Additionally, it is crucial to be detailed and specific about the action steps required to achieve the goals. Regularly assessing progress and updating action plans is also recommended, with a weekly assessment being a good starting point. Dopamine plays a role in motivation for goal pursuit, and understanding its function c

=== Calling Function ===
Calling function: diet_nutrition_vector_tool with args: {
  "input": "Ketogenic diet"
}


message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=43 request_id=1cca93a426da28d7f820d1f1a276a6da response_code=200
body='{\n  "object": "list",\n  "data": [\n    {\n      "object": "embedding",\n      "index": 0,\n      "embedding": "2VBavNMAibxew1U8OcPeOgN42bqKFaA8nclHvBhsUb36aZm8+ZJivF/ZmzzEcec8Rv0qvAXh8jo2BrG8/SZHO+keCz05hZC8eN5VPOzbODvS6kK8t89CO46phDxjgoW8LrTdu9l65DvZ/QY8LUvEvFSLC7ziIMy8tOgKPVBff7ukL9+7ztkAPK4+YDuVJCG8NcchO68AEjxwTfM881ZVvHUMIzz1lqU8+rxsPJB6djspZIy8lQ+cu6atfTuELOY7e0gwPMP0CTxA66c84HijPLPThbyPO2c8lni1vDYbtjtQSnq70z5Xu7A/ITxcRbe8undrPHiLAjy7jbG8p9hIPJO7hzwgvqS7UTY2vASi4zxf2Ru8hQMdPONf2zv6fp489sCvvBnA5bv3FEQ7qe6OvD6r17uFgPq73Lq0O4QsZrsLtac8j+gTvN5NWLybnvw8+SoKPAQ6i7sutF28poS0O3je1TrvRZO8fwafuyRnDjw8LTk8bnyBPGroHL0TMYU8wvPIu1kFZzyzEVS8Fy1CvAz0NrzFSB49WzCyO5GQvLxq0xe9DUhLvFpEdrtecAI7HWlPPFxavLyu64y8LXVOPebJtTsQ8PO8bGa7PEJUQTxDVQI9SXyKu46pBLzjX9u8D7HkPNaoMTuuFFY8GZbbuj2BzTvgeKM7JLrhvMkGDb34aFi85HTgvLQSFT05rlm7CnYYuQS3aDwVr6O8k7sHPSqOlr

Got output: The ketogenic diet is a dietary approach that involves reducing carbohydrate consumption and increasing fat intake. It is known to induce a metabolic state called ketosis, where the body primarily uses ketones, produced from fat breakdown, as a source of energy instead of glucose. The ketogenic diet has been used to control epilepsy in children and adolescents, and it has shown cognitive improvement in some individuals. It is believed that the benefits of the ketogenic diet extend beyond just the presence of ketones, as it also involves lowering glucose levels, improving insulin signaling, and promoting mitochondrial biogenesis. However, the effects of exogenous ketones, such as ketone salts or esters, as a substitute for the ketogenic diet are still uncertain and require further research.


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=27144 request_id=3255b551db1f0c627d9e9f2799d8a477 response_code=200
body='{\n  "id": "chatcmpl-83kxL77S5YBq0AFqmWAxZU5GimZ3K",\n  "object": "chat.completion",\n  "created": 1695906175,\n  "model": "gpt-3.5-turbo-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "The ketogenic diet, often referred to as the keto diet, is a low-carbohydrate, high-fat diet that has gained popularity for its potential health benefits. Here are some key points to know about the ketogenic diet:\\n\\n1. Macronutrient Composition: The ketogenic diet is characterized by a high fat, moderate protein, and very low carbohydrate intake. Typically, the macronutrient breakdown is around 70-75% fat, 20-25% protein, and 5-10% carbohydrates.\\n\\n2. Inducing Ketosis: By significantly reducing carbohydrate intake, the body is forced to enter a metabolic state cal

In [16]:
response.sources

[ToolOutput(content='The ketogenic diet is a dietary approach that involves reducing carbohydrate consumption and increasing fat intake. It is known to induce a metabolic state called ketosis, where the body primarily uses ketones, produced from fat breakdown, as a source of energy instead of glucose. The ketogenic diet has been used to control epilepsy in children and adolescents, and it has shown cognitive improvement in some individuals. It is believed that the benefits of the ketogenic diet extend beyond just the presence of ketones, as it also involves lowering glucose levels, improving insulin signaling, and promoting mitochondrial biogenesis. However, the effects of exogenous ketones, such as ketone salts or esters, as a substitute for the ketogenic diet are still uncertain and require further research.', tool_name='diet_nutrition_vector_tool', raw_input={'input': 'Ketogenic diet'}, raw_output=Response(response='The ketogenic diet is a dietary approach that involves reducing car